# 🖥️ 06: Filesystem and Code Execution

Learn how to give your LLM the ability to execute Python code and perform file operations, enabling it to compute results and persist data.

## 📋 Learning Objectives

By the end of this notebook, you will be able to:

- [ ] Execute Python code dynamically using the `execute_python` tool
- [ ] Perform file operations with the `filesystem_operation` tool
- [ ] Understand working directory concepts for file paths
- [ ] Separate concerns: computation vs. I/O operations
- [ ] Handle security considerations for code execution
- [ ] Build a file-based counter using both tools
- [ ] Understand when to use each tool appropriately

## 🎯 Prerequisites

- Completed notebook 05 (Custom Tools)
- Understanding of tool registration and usage
- Basic Python file operations knowledge
- Awareness of code execution security implications

## ⏱️ Estimated Time: 15 minutes

## ⚠️ Security Warning

**IMPORTANT:** The `execute_python` tool executes arbitrary Python code. This is powerful but potentially dangerous:

- Only use in trusted environments
- Never expose to untrusted users without sandboxing
- Be aware that code can modify files, network, etc.
- Consider using restricted execution environments for production

For learning purposes in this notebook, we're using it in a controlled manner.

## 1️⃣ The execute_python Tool

The `execute_python` tool runs Python code dynamically and returns the results.

In [14]:
from dotenv import load_dotenv
import os

load_dotenv()

from local_llm_sdk import LocalLLMClient

# Create client and register all built-in tools
client = LocalLLMClient(
    base_url=os.getenv("LLM_BASE_URL"),
    model=os.getenv("LLM_MODEL")
)

# Register built-in tools (execute_python, filesystem_operation, etc.)
client.register_tools_from(None)

print("✅ Registered tools:")
for tool_name in client.tools.list_tools():
    print(f"  - {tool_name}")

Let's execute some Python code:

In [15]:
# Simple calculation
response = client.chat("Use Python to calculate the factorial of 10")
print("Factorial calculation:")
print(response)
client.print_tool_calls()

Factorial calculation:
The factorial of 10 is calculated to be:

\[
\boxed{3628800}
\]

🔧 Tool Execution Summary (1 call):
  [1] execute_python(code=def factorial(n):\n    if n == 0:\n        return 1\n    else:\n        return n * factorial(n-1)\n\nresult = factorial(10)) → captured_result=3628800



**💡 What happened:**
1. LLM recognized it needs to execute Python code
2. It called `execute_python` with code like: `import math; math.factorial(10)`
3. The code was executed in a Python interpreter
4. The result was returned to the LLM
5. The LLM formatted a response

## 2️⃣ More Complex Computations

The execute_python tool can handle complex calculations and algorithms.

In [16]:
# Statistical analysis
response = client.chat(
    "Calculate the mean, median, and standard deviation of these numbers: "
    "[23, 45, 12, 67, 89, 34, 56, 78, 90, 21]"
)
print("Statistics:")
print(response)
client.print_tool_calls()

print("\n" + "="*70 + "\n")

# Generate sequence
response = client.chat(
    "Generate the first 15 Fibonacci numbers using Python"
)
print("Fibonacci sequence:")
print(response)
client.print_tool_calls()

print("\n" + "="*70 + "\n")

# String manipulation
response = client.chat(
    "Use Python to find all palindromes in this list: "
    "['hello', 'racecar', 'python', 'level', 'world', 'madam']"
)
print("Palindrome finder:")
print(response)
client.print_tool_calls()

Statistics:
The mean is approximately 52.7, the median is 49.5, and the standard deviation is approximately 27.13 for the given numbers [23, 45, 12, 67, 89, 34, 56, 78, 90, 21].

🔧 Tool Execution Summary (1 call):
  [1] execute_python(code=import numpy as np
numbers = [23, 45, 12, 67, 89, 34, 56, 78, 90, 21]
mean = np.mean(numbers)
median = np.median(numbers)
std_dev = np.std(numbers)
(mean, median, std_dev)) → success=True



Fibonacci sequence:
The first 15 Fibonacci numbers are:

\[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377\]

🔧 Tool Execution Summary (1 call):
  [1] execute_python(code=def generate_fibonacci(n):
    fibonacci_sequence = []
    a, b = 0, 1
    for _ in range(n):
        fibonacci_sequence.append(a)
        a, b = b, a + b
    return fibonacci_sequence

# Generate the first 15 Fibonacci numbers
fibonacci_numbers = generate_fibonacci(15)
fibonacci_numbers) → success=True



Palindrome finder:
The palindromes in the list are: ['racecar', 'level', 'madam']



## 3️⃣ The filesystem_operation Tool

The `filesystem_operation` tool allows reading, writing, and listing files.

In [17]:
import os
import tempfile

# Create a temporary directory for our examples
temp_dir = tempfile.mkdtemp()
print(f"📁 Working directory: {temp_dir}\n")

# Write a file
response = client.chat(
    f"Create a file called 'hello.txt' in {temp_dir} with the content: 'Hello from the LLM!'"
)
print("File creation:")
print(response)
client.print_tool_calls()

📁 Working directory: /tmp/tmp0yo19dv9

File creation:
The file 'hello.txt' has been created in '/tmp/tmp0yo19dv9' with the content: 'Hello from the LLM!'. The size of the file is 19 bytes.

🔧 Tool Execution Summary (2 calls):
  [1] filesystem_operation(operation=create_dir, path=/tmp/tmp0yo19dv9) → success=True
  [2] filesystem_operation(operation=write_file, path=/tmp/tmp0yo19dv9/hello.txt, content=Hello from the LLM!) → success=True



Read the file back:

In [18]:
# Read the file
response = client.chat(f"Read the contents of {temp_dir}/hello.txt")
print("File contents:")
print(response)

File contents:
The contents of the file /tmp/tmp0yo19dv9/hello.txt are:

Hello from the LLM!


List files in directory:

In [19]:
# List directory
response = client.chat(f"List all files in {temp_dir}")
print("Directory listing:")
print(response)

Directory listing:
Here are the files in the directory `/tmp/tmp0yo19dv9`:

- `hello.txt` (Size: 19 bytes)


**💡 filesystem_operation supports:**
- `read`: Read file contents
- `write`: Create or overwrite files
- `list`: List directory contents
- `append`: Add to existing files (if supported)
- `delete`: Remove files (if supported)

## 4️⃣ Combining Computation and File I/O: Two Approaches

Let's combine `execute_python` (for computation) and `filesystem_operation` (for storage) using two different patterns:

1. **Manual Multi-Step**: We orchestrate the tools ourselves (more control)
2. **Agent Approach**: The LLM orchestrates automatically (more autonomous)

This comparison will help you understand when to use each approach!

In [21]:
### Manual Multi-Step Approach

# When tasks are complex, it's better to break them into explicit steps
# Step 1: Generate numbers and calculate statistics

history = []

print("📊 Step 1: Generate data and calculate statistics\n")
response1, history = client.chat_with_history(
    "Use execute_python to generate a list of 20 random numbers between 1 and 100. "
    "Calculate their mean and median. "
    "Store the results in variables called: numbers, mean_value, median_value",
    tool_choice="required",
    history=history
)
print(response1)
client.print_tool_calls()

print("\n" + "="*70 + "\n")

# Step 2: Save the results to a file
print("💾 Step 2: Save results to file\n")
response2, history = client.chat_with_history(
    f"Use filesystem_operation to write a file at {temp_dir}/data_analysis.txt"
    f"with this content:\n"
    f"Random Numbers: [list of 20 numbers]\n"
    f"Mean: [calculated mean]\n"
    f"Median: [calculated median]\n"
    f"\nUse the data from the previous calculation.",
    tool_choice="required",
    history=history
)
print(response2)
client.print_tool_calls()

print("\n" + "="*70)
print("\n💡 Notice: We manually orchestrated 2 tool calls in sequence")
print("   This works, but requires us to manage the workflow ourselves")

📊 Step 1: Generate data and calculate statistics

Here is the list of 20 random numbers between 1 and 100 along with their mean and median:

```python
numbers = [6, 58, 82, 72, 5, 37, 81, 40, 10, 41, 25, 14, 12, 72, 78, 9, 35, 84, 12, 27]
mean_value = 40
median_value = 36.0
```

🔧 Tool Execution Summary (1 call):
  [1] execute_python(code=import random
import statistics

# Generate a list of 20 random numbers between 1 and 100
numbers = [random.randint(1, 100) for _ in range(20)]
mean_value = statistics.mean(numbers)
median_value = statistics.median(numbers)

# Prepare the results as a dictionary to return
result = {"numbers": numbers, "mean_value": mean_value, "median_value": median_value}
result, timeout=10) → captured_result={'numbers': [6, 58, 82, 72, 5, 37, 81, 40, 10, 41, 25, 14, 12, 72, 78, 9, 35, 84, 12, 27], 'mean_value': 40, 'median_value': 36.0}



💾 Step 2: Save results to file

The file has been successfully created at `/tmp/tmp0yo19dv9/data_analysis.txt` with the followin

### The Agent Approach (Preview)

**What if the LLM could automatically figure out the steps and orchestrate the tools itself?**

That's exactly what **agents** do! Let's see the same task with an agent:

In [22]:
# Use the react() method for autonomous multi-step execution
print("🤖 Agent Approach: Autonomous execution\n")

result = client.react(
    f"Generate a list of 20 random numbers between 1 and 100, "
    f"calculate their mean and median, "
    f"then save all the data to {temp_dir}/data_analysis.txt",
    max_iterations=10
)

print(f"Status: {result.status}")
print(f"Iterations: {result.iterations}")
print(f"\nFinal response:\n{result.final_response}")

print("\n" + "="*70)
print("\n🎯 Key Difference:")
print("   Manual: We specified 2 separate steps")
print("   Agent:  It figured out the steps automatically!")
print("\n💡 This is the power of ReACT agents - they reason about what")
print("   tools to use and when to use them, handling the orchestration for you.")
print("\n➡️  Learn more about agents in the next notebook: 07-react-agents.ipynb")

🤖 Agent Approach: Autonomous execution

Status: AgentStatus.ERROR
Iterations: 0

Final response:



🎯 Key Difference:
   Manual: We specified 2 separate steps
   Agent:  It figured out the steps automatically!

💡 This is the power of ReACT agents - they reason about what
   tools to use and when to use them, handling the orchestration for you.

➡️  Learn more about agents in the next notebook: 07-react-agents.ipynb


In [ ]:
# Verify the file was created
import os
file_path = f"{temp_dir}/data_analysis.txt"

if os.path.exists(file_path):
    print(f"✅ File created successfully!")
    print(f"\n📄 Contents of {file_path}:\n")
    with open(file_path, 'r') as f:
        print(f.read())
else:
    print(f"❌ File not found: {file_path}")

✅ File created successfully!

📄 Contents of /tmp/tmp1lb4xc4f/data_analysis.txt:

Random Numbers: [56, 21, 87, 34, 65, 23, 91, 12, 45, 78, 32, 60, 89, 10, 35, 72, 44, 29, 53, 90]
Mean: 49.6
Median: 49.0


In [24]:
# Use the react() method for autonomous multi-step execution
print("🤖 Agent Approach: Autonomous execution\n")

result = client.react(
    f"Generate a list of 20 random numbers between 1 and 100, "
    f"calculate their mean and median, "
    f"then save all the data to {temp_dir}/data_analysis.txt",
    max_iterations=10
)

print(f"Status: {result.status}")
print(f"Steps taken: {result.iterations}")
print(f"\nFinal response:\n{result.final_response}")

print("\n" + "="*70)
print("\n🎯 Key Difference:")
print("   Manual: We specified 2 separate steps")
print("   Agent:  It figured out the steps automatically!")
print("\n💡 This is the power of ReACT agents - they reason about what")
print("   tools to use and when to use them, handling the orchestration for you.")
print("\n➡️  Learn more about agents in the next notebook: 07-react-agents.ipynb")

🤖 Agent Approach: Autonomous execution

ReACT Agent: Starting task
Max iterations: 10
Task: Generate a list of 20 random numbers between 1 and 100, calculate their mean and median, then save a...


Iteration 1/10
----------------------------------------
Response: [TOOL_CALLS]execute_python[ARGS]{"code": "import random\nrandom_numbers = [random.randint(1, 100) for _ in range(20)]\nmean_value = sum(random_numbers...
Tools used: 1
  - execute_python

Iteration 2/10
----------------------------------------
Response: Got the numbers: [87, 32, 94, 23, 12, 45, 67, 56, 78, 19, 30, 41, 52, 63, 74, 85, 96, 10, 2, 3] with mean: 45.0 and median: 41.0.

Next, I'll save thi...
Tools used: 1
  - execute_python

Iteration 3/10
----------------------------------------
Response: [TOOL_CALLS]filesystem_operation[ARGS]{"operation": "read_file", "path": "/tmp/tmp0yo19dv9/data_analysis.txt"}
Tools used: 2
  - filesystem_operation
  - TASK_COMPLETE, the data has been saved to /tmp/tmp0yo19dv9/data_analysis.t

In [25]:
# Read the analysis file
response = client.chat(f"Read {temp_dir}/data_analysis.txt")
print("Analysis results:")
print(response)

Analysis results:
Here is the content of the file /tmp/tmp0yo19dv9/data_analysis.txt:

Random numbers: [42, 87, 3, 56, 91, 14, 65, 22, 78, 33, 5, 89, 27, 45, 70, 9, 31, 62, 18, 54]
Mean: 42.0
Median: 42


## 5️⃣ Tool Separation: When to Use Each

Understanding when to use `execute_python` vs `filesystem_operation`:

### execute_python: For Computation

Use when you need to:
- Perform calculations
- Run algorithms
- Process data structures
- Use Python libraries
- Generate sequences or patterns

**Example tasks:**
- Sort an array
- Calculate statistics
- Parse dates
- Generate random data
- Apply mathematical formulas

In [26]:
# Good use of execute_python
response = client.chat(
    "Sort these numbers in descending order and show every 3rd element: "
    "[45, 12, 78, 23, 89, 34, 56, 67, 90, 21, 43, 65, 87, 32, 54]"
)
print("Computation example:")
print(response)

Computation example:
The numbers sorted in descending order and every 3rd element are as follows:

Sorted Numbers: [90, 89, 87, 78, 67, 65, 56, 54, 45, 43, 34, 32, 23, 21, 12]

Every 3rd Element: [87, 65, 54, 43, 32]


### filesystem_operation: For I/O

Use when you need to:
- Read data from files
- Save results to disk
- List directory contents
- Persist state between conversations
- Load configuration

**Example tasks:**
- Save report to file
- Read CSV data
- Create log entries
- Store user preferences
- Load templates

In [27]:
# Good use of filesystem_operation
response = client.chat(
    f"Create a todo list file at {temp_dir}/todos.txt with these items:\n"
    "1. Learn about filesystem tools\n"
    "2. Practice code execution\n"
    "3. Build something cool"
)
print("File I/O example:")
print(response)

File I/O example:
The todo list has been created at the specified location. Here are the contents of `/tmp/tmp0yo19dv9/todos.txt`:
```
1. Learn about filesystem tools
2. Practice code execution
3. Build something cool
```


## 6️⃣ Working Directory Considerations

File paths can be relative or absolute. Understanding the working directory is important.

In [28]:
# Check current working directory
response = client.chat("Use Python to print the current working directory")
print("Current working directory:")
print(response)

print("\n💡 Tip: Always use absolute paths when working with files to avoid confusion!")
print(f"   Example absolute path: {temp_dir}/myfile.txt")
print(f"   Example relative path: myfile.txt (depends on CWD)")

Current working directory:
The current working directory is `/tmp/python_exec_ujzr1k2a`.

💡 Tip: Always use absolute paths when working with files to avoid confusion!
   Example absolute path: /tmp/tmp0yo19dv9/myfile.txt
   Example relative path: myfile.txt (depends on CWD)


## 🏋️ Exercise: File-Based Counter

**Challenge:** Build a persistent counter that:
1. Stores its value in a file called `counter.txt`
2. Reads the current value from the file (defaults to 0 if file doesn't exist)
3. Increments the counter by a specified amount
4. Saves the new value back to the file
5. Returns the old value, new value, and increment amount

Requirements:
- Use `filesystem_operation` to read and write the file
- Use `execute_python` if needed for calculations
- Handle the case where the file doesn't exist yet
- Test incrementing multiple times

Try it yourself first!

In [ ]:
# Your code here:



<details>
<summary>Click to see solution</summary>

```python
# Solution: File-based persistent counter

import os

counter_file = os.path.join(temp_dir, "counter.txt")

print("🔢 File-Based Counter Application\n")
print("="*70 + "\n")
print(f"Counter file: {counter_file}\n")

# Initialize counter (first use)
print("Step 1: Initialize counter")
response = client.chat(
    f"Check if {counter_file} exists. If not, create it with value '0'. "
    f"If it exists, read its current value."
)
print(response)
print()

# Increment by 5
print("Step 2: Increment by 5")
response = client.chat(
    f"Read the number from {counter_file}, add 5 to it, "
    f"save the new value back to the file, and tell me both the old and new values."
)
print(response)
print()

# Increment by 10
print("Step 3: Increment by 10")
response = client.chat(
    f"Read the number from {counter_file}, add 10 to it, "
    f"save the new value back to the file, and tell me both the old and new values."
)
print(response)
print()

# Increment by 3
print("Step 4: Increment by 3")
response = client.chat(
    f"Read the number from {counter_file}, add 3 to it, "
    f"save the new value back to the file, and tell me both the old and new values."
)
print(response)
print()

# Final check
print("Step 5: Verify final value")
response = client.chat(f"What is the current value in {counter_file}?")
print(response)
print()

print("="*70)
print("\n✅ Counter successfully persisted across multiple operations!")
print(f"   Expected final value: 18 (0 + 5 + 10 + 3)")
```
</details>

In [ ]:
# Solution cell (run this to see the answer)
counter_file = os.path.join(temp_dir, "counter.txt")

print("🔢 File-Based Counter Application\n")
print("="*70 + "\n")
print(f"Counter file: {counter_file}\n")

# Initialize
print("Step 1: Initialize counter")
response = client.chat(
    f"Check if {counter_file} exists. If not, create it with value '0'. "
    f"If it exists, read its current value."
)
print(response)
print()

# Increment by 5
print("Step 2: Increment by 5")
response = client.chat(
    f"Read the number from {counter_file}, add 5 to it, "
    f"save the new value back to the file, and tell me both the old and new values."
)
print(response)
print()

# Increment by 10
print("Step 3: Increment by 10")
response = client.chat(
    f"Read the number from {counter_file}, add 10 to it, "
    f"save the new value back to the file, and tell me both the old and new values."
)
print(response)
print()

# Increment by 3
print("Step 4: Increment by 3")
response = client.chat(
    f"Read the number from {counter_file}, add 3 to it, "
    f"save the new value back to the file, and tell me both the old and new values."
)
print(response)
print()

# Final check
print("Step 5: Verify final value")
response = client.chat(f"What is the current value in {counter_file}?")
print(response)
print()

print("="*70)
print("\n✅ Counter successfully persisted across multiple operations!")
print(f"   Expected final value: 18 (0 + 5 + 10 + 3)")

## 🧹 Cleanup

Let's clean up our temporary directory:

In [29]:
import shutil

# Remove temporary directory
shutil.rmtree(temp_dir)
print(f"✅ Cleaned up temporary directory: {temp_dir}")

✅ Cleaned up temporary directory: /tmp/tmp0yo19dv9


## ⚠️ Common Pitfalls

### 1. Executing Unsafe Code
```python
# ❌ DANGEROUS: Never execute untrusted code
user_code = input("Enter code: ")  # User could enter malicious code!
response = client.chat(f"Execute this Python: {user_code}")

# ✅ Good: Validate and sanitize, or use sandboxing
# In production, use restricted execution environments
```

### 2. Using Relative Paths
```python
# ⚠️ Warning: Relative paths depend on working directory
response = client.chat("Read myfile.txt")  # Where is myfile.txt?

# ✅ Good: Use absolute paths
import os
file_path = os.path.abspath("myfile.txt")
response = client.chat(f"Read {file_path}")
```

### 3. Not Handling File Errors
```python
# ❌ Bad: Assuming file exists
response = client.chat(f"Read {nonexistent_file}")
# Will error if file doesn't exist

# ✅ Good: Check existence first or handle errors
response = client.chat(
    f"Check if {file_path} exists. If yes, read it. If no, say file not found."
)
```

### 4. Forgetting About Permissions
```python
# ⚠️ Warning: May not have write permissions
response = client.chat("Write to /root/protected.txt")
# Could fail due to permissions

# ✅ Good: Use directories you own
import tempfile
temp_dir = tempfile.mkdtemp()
response = client.chat(f"Write to {temp_dir}/file.txt")
```

### 5. Mixing Concerns
```python
# ⚠️ Suboptimal: Using execute_python for file I/O
response = client.chat(
    "Use Python to open 'data.txt', read it, and print contents"
)
# Works, but filesystem_operation is designed for this

# ✅ Better: Use the right tool for the job
response = client.chat("Read the file 'data.txt'")
# filesystem_operation handles this directly
```

## 🎓 What You Learned

✅ **execute_python**: Run Python code dynamically for computations

✅ **filesystem_operation**: Read, write, list files for I/O operations

✅ **Tool Separation**: Use execute_python for compute, filesystem_operation for I/O

✅ **Working Directories**: Understand absolute vs relative paths

✅ **Combining Tools**: Chain computation and storage for complete workflows

✅ **Security**: Be aware of code execution risks

✅ **Error Handling**: Check file existence, handle permissions

✅ **Best Practices**: Use absolute paths, validate inputs, choose appropriate tools

## 🎉 Congratulations!

You've completed the core tutorial series for the Local LLM SDK!

### What You've Mastered:

1. **Basic Chat**: Simple interactions, system prompts, temperature control
2. **Conversation History**: Multi-turn conversations with context
3. **Built-in Tools**: Using pre-made tools (calculator, text transformer, etc.)
4. **Custom Tools**: Creating your own tools with the @tool decorator
5. **Filesystem & Code**: Executing Python and managing files

### 🚀 What's Next?

Now that you have the foundations, you can:

**Build Real Applications:**
- Create a chatbot with tool capabilities
- Build a data analysis assistant
- Make an automation tool for repetitive tasks
- Develop a coding helper that executes and tests code

**Explore Advanced Topics:**
- Streaming responses for real-time output
- Error handling and retry logic
- Tool chaining for complex workflows
- Integration with external APIs and services
- Production deployment and security

**Check Out the SDK Documentation:**
- [README.md](../README.md) - Project overview and quick start
- [local_llm_sdk/](../local_llm_sdk/) - Source code with detailed docstrings
- `.documentation/` - Research and API comparisons

### 💬 Questions or Issues?

If you encounter any problems or have questions:
1. Check the SDK source code for implementation details
2. Review the documentation in `.documentation/`
3. Test with LM Studio server logs for debugging
4. Experiment in these notebooks to understand behavior

### 🌟 Keep Learning!

The best way to master the SDK is to build something. Start small, experiment often, and gradually increase complexity.

Happy coding with Local LLM SDK!